In [1]:
from keplergl import KeplerGl
import pandas as pd

In [2]:
'''
Define paths and parameters
'''
DATA_PATH = './17_18.csv'

GEO_DATA_PATH = './country_lat_long.csv'

In [3]:
# load dataset
DATASET = pd.read_csv(DATA_PATH, sep=';')

# replace Datum Uhrzeit with proper timestamp as index
DATASET['timestamp'] = DATASET.Datum.map(str) + ' ' + DATASET.Uhrzeit
DATASET = DATASET.drop(['Uhrzeit', 'Datum'], axis=1)
DATASET.set_index('timestamp', inplace=True)
# transform to time series
DATASET.index = pd.to_datetime(DATASET.index)
DATASET = DATASET.apply(pd.to_numeric, errors='coerce')
DATASET.fillna(0, inplace=True)

# load geo data
GEO_DATA = pd.read_csv(GEO_DATA_PATH, sep=';')
print(GEO_DATA)

       country       long        lat
0    nl_import  52.208448   6.804434
1    nl_export  51.938339   6.628553
2    lu_import  49.945034   6.233296
3    lu_export  49.604481   6.408965
4    fr_import  49.089157   7.375149
5    fr_export  48.684614   7.946076
6    ch_import  47.629306   8.165664
7    ch_export  47.629306   9.009500
8    au_import  47.599681  12.206069
9    au_export  47.954065  13.041866
10   cz_import  49.290202  12.783043
11   cz_export  50.001560  12.299951
12   pl_import  52.342891  14.581793
13   pl_export  52.019543  14.713545
14   dk_import  54.769801   8.740771
15   dk_export  54.769801   9.487368
16   sw_import  55.098016  13.130464
17   sw_export  55.072864  13.525799
18  net_export  51.228388   9.707071


In [6]:
'''
Define data operation functions
'''

def shrinkIntervalMean(data, interval):
    '''
    returns the resampled dataset with the mean of the timestamp values in the given interval, 
    e.g. '10H' for 10 hour intervals
    '''
    return data.resample(interval).mean()

def toKeplerData(dataset, geodata):
    '''
    returns the given time series with each column as a distinct timestamp 
    in the series with the specific lat/lang attached
    '''
    series = pd.DataFrame()
    columns = list(dataset.columns)
    for index, row in dataset.iterrows():
        for col in columns:
            geo = geodata.loc[geodata['country'] == col]
            newTimestamp = pd.DataFrame({'value':row[col],'lat':geo.iloc[0]['lat'],
                                         'long':geo.iloc[0]['long']},index=[index])
            series = series.append(newTimestamp)

    return series

'''
Test functions
'''
#DATASET = shrinkTimeGran(DATASET, '5H')
#print(toKeplerData(DATASET, GEO_DATA).head(5))

             value       lat       long
2017-01-01  9.3288  9.707071  51.228388
2017-01-01  1.3218  6.628553  51.938339
2017-01-01 -2.0000  6.804434  52.208448
2017-01-01  1.2000  9.009500  47.629306
2017-01-01  0.0000  8.165664  47.629306


In [16]:
'''
Data preparation
'''

# reduce time series granularity to 1 Day
DATASET = shrinkIntervalMean(DATASET, '23H')

toKeplerData(DATASET, GEO_DATA).head(90).to_csv(r'geo_data_23H.csv')

In [18]:
'''
- Define Geo Data Location for data visualization
- Welches Format brauchen die Geo-Daten?
- Wo bekommen wir die Daten her?
- Wie macht man die Balken und Blasen mit kepler?

https://www.latlong.net
'''

map_1 = KeplerGl(height=500)

import_export_data = pd.read_csv("./geo_data_23H.csv", sep=",")
map_1.add_data(data=import_export_data, name='Strom Import/Export')

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Strom Import/Export': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,…